In [1]:
import pandas as pd
import spacy
import gensim
import numpy as np
import re
from gensim.models import LdaModel,HdpModel
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
!pip install pyLDAvis
import pyLDAvis.gensim
!pip install "pandas<2.0.0"
!pip install fasttext-langdetect
from ftlangdetect import detect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 86.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicor

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext-langdetect: filename=fasttext_langdetect-1.0.5-py3-none-any.whl size=7503 sha256=92cb327c941ddd0411b1ec65d740cb9893910ab4865ee13cadc9ea5aad321f09
  Stored in directory: /root/.cache/pip/wheels/e6/5b/5d/47e2fd5c2ff1028722739ce35f365e8f6eeb89ec97aa63e621
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199771 sha256=4612e43affcfe7cded738b6e3a535c2eff940efccb4241a6175d6ed9a071f40d
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext-langdetect fasttext


In [2]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
path = "/content/drive/MyDrive/Uppsala University/Internship/unicef"
narrative_file = path + "/RAM3_2022.csv"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pexpect/popen_spawn.py:60: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  self._read_thread.setDaemon(True)


Mounted at /content/drive/


Reading Data

In [3]:
narrative_df = pd.read_csv(narrative_file)
narrative_df.drop(columns=['REGION_NAME','NarrativeTitle'],inplace=True)
print(narrative_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


         BUSINESS_AREA_NAME                                      NarrativeText
0                  Cambodia  Identification of vulnerable populations for p...
1                     China  It is important to demonstrate the effectivene...
2      DP Republic of Korea  Under the challenging operational circumstance...
3           EAPRO, Thailand  Partnerships and Engagement  \r\nEAPRO continu...
4    Fiji (Pacific Islands)  The COVID-19 pandemic has been a major catalys...
..                      ...                                                ...
147     Sao Tome & Principe  Adequate financing and efficient public spendi...
148                 Senegal  In 2022, UNICEF Senegal continued to spearhead...
149            Sierra Leone  Following three years of implementation, an ev...
150                    Togo  L’approche périmètre brachial, une lueur d’esp...
151          WCARO, Senegal  Creation of regional repository for KRC accele...

[152 rows x 2 columns]


Cleaning Data

In [4]:
# remove non-en narratives
narrative_df['NarrativeText'] = narrative_df['NarrativeText'].apply(lambda x: x.replace("\n"," "))
narrative_df['is_en'] = narrative_df['NarrativeText'].apply(lambda x: detect(text = x, low_memory = True))
print(narrative_df.head(5))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


       BUSINESS_AREA_NAME                                      NarrativeText  \
0                Cambodia  Identification of vulnerable populations for p...   
1                   China  It is important to demonstrate the effectivene...   
2    DP Republic of Korea  Under the challenging operational circumstance...   
3         EAPRO, Thailand  Partnerships and Engagement  \r EAPRO continue...   
4  Fiji (Pacific Islands)  The COVID-19 pandemic has been a major catalys...   

                                         is_en  
0  {'lang': 'en', 'score': 0.9368439316749573}  
1  {'lang': 'en', 'score': 0.9242146015167236}  
2   {'lang': 'en', 'score': 0.940678596496582}  
3  {'lang': 'en', 'score': 0.9201379418373108}  
4  {'lang': 'en', 'score': 0.9209935665130615}  


In [5]:
# remove non-eng narratives.
print(narrative_df.size)
valid_narr = [ True if ( x['lang'] =='en' and x['score']>=0.85) else False for x in narrative_df['is_en']]
# print(narrative_df[valid_narr])
narrative_df = narrative_df[valid_narr]
print(narrative_df.size)

456
432


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
nlp = spacy.load('en_core_web_sm')
docs = []
for narr in list(narrative_df['NarrativeText']):
  docs.append(nlp(narr))
print(docs[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
#remove stop words
country_narr, narrs = [], []
for doc in docs:
  for token in doc:
    if token.text != '\r\n' and token.text != '\r\n\r\n' and token.text !='\xa0' and not token.is_stop and not token.is_punct and not token.like_num :
        country_narr.append(token.lemma_)
  narrs.append(country_narr)
  country_narr = []

#consider bigrams.
bigram = gensim.models.phrases.Phrases(narrs)
narrs = [bigram[line] for line in narrs]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
dictionary = Dictionary(narrs)
corpus = [dictionary.doc2bow(narr) for narr in narrs]
gensim.corpora.MmCorpus.serialize('MmCorpusTest.mm', corpus) #save corpus

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
pattern = re.compile(r"[\xa0|\r]+")
del_ids = [k for k,v in dictionary.items() if re.match(pattern, v)]
del_ids += [k for k,v in dictionary.items() if v in ['child','UNICEF','unicef','support','programme']]
del_ids += [k for k,v in dictionary.items() if v in ['child','UNICEF','unicef','support','programme', 'need','include','system', 'work',
                                                    'strengthen', 'service', 'community','approach','result','development']]
# remove unwanted word ids from the dictionary in place
dictionary.filter_tokens(bad_ids=del_ids)
# remove unwanted wordids from corpus
corpusMm = gensim.corpora.MmCorpus('MmCorpusTest.mm')
np_corpus = gensim.matutils.corpus2dense(corpusMm, corpusMm.num_terms, num_docs=corpusMm.num_docs).T
np_corpus = np.delete(np_corpus, del_ids, 1).T
corpus = gensim.matutils.Dense2Corpus(np_corpus)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LDA

In [10]:
lda_model = LdaModel(corpus=corpus, num_topics=15, id2word=dictionary)
lda_model.show_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(13,
  '0.005*"response" + 0.005*"provide" + 0.004*"government" + 0.004*"key" + 0.004*"partner" + 0.004*"increase" + 0.004*"country" + 0.003*"staff" + 0.003*"scale" + 0.003*"level"'),
 (9,
  '0.004*"partner" + 0.004*"capacity" + 0.004*"national" + 0.004*"level" + 0.004*"increase" + 0.004*"ensure" + 0.004*"strategy" + 0.003*"provide" + 0.003*"develop" + 0.003*"school"'),
 (1,
  '0.006*"develop" + 0.005*"education" + 0.004*"provide" + 0.004*"ensure" + 0.004*"new" + 0.004*"health" + 0.004*"country" + 0.004*"initiative" + 0.004*"response" + 0.004*"school"'),
 (8,
  '0.005*"increase" + 0.004*"partnership" + 0.004*"continue" + 0.004*"intervention" + 0.004*"key" + 0.004*"develop" + 0.004*"focus" + 0.004*"learn" + 0.004*"partner" + 0.003*"new"'),
 (3,
  '0.004*"partnership" + 0.004*"level" + 0.004*"provide" + 0.004*"innovation" + 0.003*"continue" + 0.003*"education" + 0.003*"sector" + 0.003*"opportunity" + 0.003*"ensure" + 0.003*"increase"'),
 (6,
  '0.004*"ensure" + 0.004*"partnership" + 0.0

In [11]:
# hdp_model = HdpModel(corpus=corpus, id2word=dictionary)
# hdp_model.show_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.005776 -0.012288       1        1  38.564592
10    -0.006040 -0.001921       2        1  13.448382
8     -0.018176  0.010621       3        1  10.965374
1      0.000287 -0.004113       4        1   5.873601
5     -0.001870 -0.000179       5        1   5.631920
2     -0.002322 -0.003431       6        1   5.332550
0      0.003219  0.001495       7        1   4.660536
11    -0.003060 -0.000546       8        1   4.162193
6      0.000935 -0.000053       9        1   3.413120
4      0.003971 -0.005977      10        1   3.361886
13     0.004850 -0.005554      11        1   1.481313
12     0.002252  0.000994      12        1   1.149972
7      0.004001  0.003370      13        1   0.904266
3      0.013041  0.005922      14        1   0.607277
9      0.004688  0.011658      15        1   0.443018, topic_info=            Term        Freq       Total Category  logprob  loglift
198      provide  234.000000  234.000000  Default  30.0000  30.0000
179      partner  231.000000  231.000000  Default  29.0000  29.0000
163     national  259.000000  259.000000  Default  28.0000  28.0000
102       ensure  220.000000  220.000000  Default  27.0000  27.0000
601  partnership  209.000000  209.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
164          new    0.686692  192.143899  Topic15  -5.9432  -0.2148
608      process    0.598030  166.986783  Topic15  -6.0814  -0.2127
378      country    0.606432  224.133249  Topic15  -6.0675  -0.4931
414          key    0.603405  225.840937  Topic15  -6.0725  -0.5057
403   government    0.592159  194.203374  Topic15  -6.0913  -0.3736

[1224 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
7167      1  0.586670  2022–2025
5237      2  0.679422        ACT
1715      1  0.189115       ADAP
1715      2  0.189115       ADAP
1715      8  0.189115       ADAP
...     ...       ...        ...
1077     12  0.008660      youth
1077     13  0.008660      youth
1077     14  0.008660      youth
5377      1  0.482281       ywca
3344      1  0.247537       zone

[3064 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 11, 9, 2, 6, 3, 1, 12, 7, 5, 14, 13, 8, 4, 10])

# TODO
* (DONE) Remove  non-eng narrs. Use fasttext.
* (DONE) remove any token starting with '\\'. user regex.
* (DONE) create bigrams to remove de and des

